In [15]:
import pandas as pd
import psycopg2

In [16]:
caminho_do_arquivo = r"C:\Users\edmil\OneDrive\Documentos\Cursos\Python - SGBDS - Arquivos\Origem de dados\V_OCORRENCIA_AMPLA.json"
df = pd.read_json(caminho_do_arquivo, encoding='utf-8-sig')

In [17]:
df.head(3)

,Numero_da_Ocorrencia,Numero_da_Ficha,Operador_Padronizado,Classificacao_da_Ocorrência,Data_da_Ocorrencia,Hora_da_Ocorrência,Municipio,UF,Regiao,Descricao_do_Tipo,...,Lesoes_Desconhecidas_Tripulantes,Lesoes_Desconhecidas_Passageiros,Lesoes_Desconhecidas_Terceiros,Modelo,CLS,Tipo_ICAO,PMD,Numero_de_Assentos,Nome_do_Fabricante,PSSO
0,7762,201803231711488,ICON TAXI AEREO LTDA E OUTRO,Incidente,2018-03-21,20:00:00,SÃO PAULO,SP,Sudeste,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,...,0.0,0.0,0.0,AW109SP,H2T,A109,3175.0,8.0,AGUSTA,verdadeiro
1,7759,201803161337187,EJ ESCOLA DE AERONAUTICA CIVIL LTDA,Acidente,2018-03-14,15:30:00,MONTES CLAROS,MG,Sudeste,COMBUSTÍVEL,...,0.0,0.0,0.0,A152,L1P,C152,757.0,2.0,CESSNA AIRCRAFT,verdadeiro
2,7758,201802271904132,AEROTEX AVIACAO AGRICOLA LTDA.,Acidente,2018-01-26,12:20:00,INACIOLÂNDIA,GO,Centro-Oeste,EXCURSÃO DE PISTA,...,0.0,0.0,0.0,T188C,L1P,C188,1997.0,1.0,CESSNA AIRCRAFT,verdadeiro


In [18]:
colunas = ["Numero_da_Ocorrencia", "Classificacao_da_Ocorrência", "Data_da_Ocorrencia","Municipio","UF","Regiao","Nome_do_Fabricante","Modelo"]
df = df[colunas]
df.rename( columns={  'Classificacao_da_Ocorrência' : 'Classificacao_da_Ocorrencia'  } ,inplace=True )

In [19]:
df.head(3)

,Numero_da_Ocorrencia,Classificacao_da_Ocorrencia,Data_da_Ocorrencia,Municipio,UF,Regiao,Nome_do_Fabricante,Modelo
0,7762,Incidente,2018-03-21,SÃO PAULO,SP,Sudeste,AGUSTA,AW109SP
1,7759,Acidente,2018-03-14,MONTES CLAROS,MG,Sudeste,CESSNA AIRCRAFT,A152
2,7758,Acidente,2018-01-26,INACIOLÂNDIA,GO,Centro-Oeste,CESSNA AIRCRAFT,T188C


In [20]:
# Parâmetros de conexão
dbname   = 'python'
user     = 'postgres'
password = '12345'
host     = 'localhost'
port     = '5432' 

conexao = psycopg2.connect(dbname=dbname,user=user,password=password,host=host,port=port)
cursor = conexao.cursor()

#Delete base antes da carga
cursor.execute(  " delete from public.anac_mapeamento  ")

#Carga de Dados
for indice,coluna_df in df.iterrows():
    cursor.execute( """   insert into anac_mapeamento (     
                ID, 
                Classificacao_da_Ocorrencia, 
                Dt_Ocorrencia, 
                Municipio, 
                UF, 
                Regiao, 
                Fabricante,
                Modelo
            ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s) 
                
            """ ,(
                coluna_df["Numero_da_Ocorrencia"],
                coluna_df["Classificacao_da_Ocorrencia"],
                coluna_df["Data_da_Ocorrencia"],
                coluna_df["Municipio"],
                coluna_df["UF"],
                coluna_df["Regiao"],
                coluna_df["Nome_do_Fabricante"],
                coluna_df["Modelo"]
            )                
            )

conexao.commit() 
cursor.close()
conexao.close()